In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
# %matplotlib notebook


In [11]:
housedata =  pd.read_csv('data/lianjia100_clean.csv')
park = pd.read_csv('data/parks-qingdao.csv')
list(housedata.columns)

['标题',
 '挂牌价格/万',
 '元/平米',
 'tags',
 '小区名',
 '几室几厅',
 '总面积',
 '装修情况',
 '有无电梯',
 '别墅',
 '朝向',
 '区域',
 '楼层高低',
 '楼共有几层',
 '建造年份',
 '建筑形式',
 '几人关注',
 '带看次数',
 '多久以前发布',
 '几月以前发布',
 '几年以前发布',
 '几天以前发布']

In [17]:
housedata['区域'].unique()

array([' 即墨', ' 香江路', ' 薛家岛', ' 李沧万达', ' 唐岛湾', ' 夏庄', ' 女姑', ' 石老人',
       ' 胶南', ' 四方实验小区', ' 错埠岭', ' 流亭', ' 新都心', ' 下王埠', ' 八大湖', ' 北仲',
       ' 十梅庵', ' 国信体育场', ' 台东', ' 东方影都', ' 镇江路', ' 理工大学', ' 海泊河', ' 团岛',
       ' 合肥路佳世客', ' 河南', ' 书院路', ' 崂山CBD', ' 金沙滩', ' 保税区', ' 北村', ' 胶州',
       ' 黄岛区政府', ' 瑞阳路', ' 中央CBD', ' 同安路', ' 枣儿山', ' 惜福镇', ' 辛安', ' 榉林公园',
       ' 远洋广场', ' 海云庵', ' 天山银海', ' 浮山后', ' 新贵都', ' 家佳源', ' 石油大学', ' 田家花园',
       ' 水清沟', ' 积米崖', ' 闫家山', ' 雕塑园', ' 大尧', ' 福州北路', ' 汽车东站', ' 河西',
       ' 农业大学', ' 福安', ' 李村公园', ' 麦岛', ' 湖岛', ' 啤酒街', ' 大沙路', ' 西镇',
       ' 沧口公园', ' 湛山', ' 北岭', ' 城阳立交', ' 绿城', ' 西姜', ' 世纪公园', ' 黄岛港',
       ' 大学路', ' 灵山卫', ' 世园会', ' 东李', ' 八大关', ' 沙岭庄', ' 天泰', ' 科技大学',
       ' 城阳实验二小', ' 东南山', ' 张村', ' 奥帆中心', ' 大港小港', ' 辛家庄', ' 五四广场',
       ' 青岛北站', ' 老鸦岭', ' 沙子口', ' 闽江路', ' 午山', ' 中山公园', ' 西韩', ' 松山后',
       ' 娄山后', ' 文登路', ' 火车站', ' 太阳城', ' 黄岛轮渡'], dtype=object)

In [13]:
park

,Unnamed: 0,address,area,city,detail,children,comment_num,detail_url,groupon_num,lat,...,overall_rating,tag,type,lat.1,lng.1,name,province,street_id,telephone,uid
0,0.0,山东省青岛市市南区文登路28号,市南区,青岛市,1,[],100,http://api.map.baidu.com/place/detail?uid=61f6...,NaN,36.067228236397,...,4.1,旅游景点;公园,scope,36.070634,120.357364,青岛中山公园,山东省,61f634f6233ddb796b49f317,(0532)82870564,61f634f6233ddb796b49f317
1,1.0,山东省青岛市崂山区东海东路60号,崂山区,青岛市,1,[],100,http://api.map.baidu.com/place/detail?uid=9875...,NaN,36.075532474175,...,3.9,旅游景点;水族馆,scope,36.075157,120.449285,青岛海昌极地海洋世界,山东省,987573fcf31545e54d2fdfdd,"(0532)89091819,(0532)85909999",987573fcf31545e54d2fdfdd
2,2.0,山东省青岛市黄岛区珠山风景区青岛森林野生动物世界,黄岛区,青岛市,1,[],100,http://api.map.baidu.com/place/detail?uid=e256...,2,35.989818539925,...,4.5,旅游景点;景点,scope,35.987489,120.064908,野生动物园,山东省,NaN,(0532)86835555,e2562cc6933366262102206e
3,3.0,青岛市市南区龙山路16号甲,市南区,青岛市,1,[],100,http://api.map.baidu.com/place/detail?uid=93df...,NaN,36.071925850051,...,4.0,旅游景点;公园,scope,36.072566,120.337993,信号山公园,山东省,93df0dc361c15c32882fb5cc,(0532)82798055,93df0dc361c15c32882fb5cc
4,4.0,山东省青岛市黄岛区漓江西路666号,黄岛区,青岛市,1,[],39,http://api.map.baidu.com/place/detail?uid=c3aa...,NaN,35.954819654932,...,4.7,旅游景点;其他,scope,35.953722,120.214328,唐岛湾,山东省,c3aab3ea072e1844912ca339,NaN,c3aab3ea072e1844912ca339
5,5.0,山东省青岛市市南区珠海支路(奥帆中心东部市立医院),市南区,青岛市,1,[],32,http://api.map.baidu.com/place/detail?uid=e56e...,NaN,36.062039032398,...,4.8,旅游景点;公园,scope,36.061247,120.412219,燕儿岛山公园,山东省,e56e9ce0a92b4fc39e82ead1,NaN,e56e9ce0a92b4fc39e82ead1
6,6.0,青岛市市南区延安一路102号,市南区,青岛市,1,[],46,http://api.map.baidu.com/place/detail?uid=5938...,NaN,36.077888419528,...,4.5,旅游景点;动物园,scope,36.076061,120.355964,青岛动物园,山东省,593850fb6e113f5cdf1e52f6,NaN,593850fb6e113f5cdf1e52f6
7,7.0,青岛市市南区福山支路24号,市南区,青岛市,1,[],23,http://api.map.baidu.com/place/detail?uid=96b6...,NaN,36.064273609561,...,4.5,旅游景点;公园,scope,36.064961,120.342928,小鱼山公园,山东省,96b672aa809e7074ce04ef17,(0532)82868532,96b672aa809e7074ce04ef17
8,8.0,山东省青岛市城阳区兴阳路318号,城阳区,青岛市,1,[],26,http://api.map.baidu.com/place/detail?uid=b057...,NaN,36.292403840596,...,4.4,旅游景点;风景区,scope,36.291942,120.401812,青岛奥林匹克雕塑文化园,山东省,b05732acfa667389018972c0,(0532)87759871,b05732acfa667389018972c0
9,9.0,兴华路33号,李沧区,青岛市,1,[],25,http://api.map.baidu.com/place/detail?uid=5b25...,NaN,36.191175697254,...,4.1,旅游景点;公园,scope,36.191082,120.395277,沧口公园,山东省,5b25f44666b358eb10293535,NaN,5b25f44666b358eb10293535
